# Importing the Essential Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#About the Dataset:

#id: unique id for a news article
#title: the title of a news article
#author: author of the news article
#text: the text of the article; could be incomplete
#label: a label that marks whether the news article is real or fake:
 #   1: Fake news
  #  0: real News

In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/FalseNewsData.csv')

In [ ]:
news_dataset.shape

(417, 102)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101
0,0,House Dem Aide: We Didnâ€™t Even See Comeyâ€™s...,Darrell Lucus,House Dem Aide: We Didnâ€™t Even See Comeyâ€™s...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Data Processing

In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id                0
title             8
author           49
text              4
label            11
               ... 
Unnamed: 97     416
Unnamed: 98     416
Unnamed: 99     416
Unnamed: 100    416
Unnamed: 101    416
Length: 102, dtype: int64

#Dealing with missing values

In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

#Made the data Concise


In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

<ipython-input-17-5db6adfd5692>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']


In [ ]:
print(news_dataset['content'])

0      Darrell Lucus House Dem Aide: We Didnâ€™t Even...
1      Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2      Consortiumnews.com Why the Truth Might Get You...
3      Jessica Purkiss 15 Civilians Killed In Single ...
4      Howard Portnoy Iranian woman jailed for fictio...
                             ...                        
412    Adam Goldman, Eric Lichtblau and Matt Apuzzo C...
413     Trump and Clinton Choice Impacting Global Mar...
414     Chinese J-20 stealth fighter debuts in skies ...
415    J.E. Dyer Spain, Malta, under U.S./UK pressure...
416    Jon Pareles Review: In Radioheadâ€™s â€˜A Moon...
Name: content, Length: 417, dtype: object


In [ ]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

      id                                              title  \
0      0  House Dem Aide: We Didnâ€™t Even See Comeyâ€™s...   
1      1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2      2                  Why the Truth Might Get You Fired   
3      3  15 Civilians Killed In Single US Airstrike Hav...   
4      4  Iranian woman jailed for fictional unpublished...   
..   ...                                                ...   
412  404  Comey Letter on Clinton Email Is Subject of Ju...   
413  405  Trump and Clinton Choice Impacting Global Markets   
414  406  Chinese J-20 stealth fighter debuts in skies o...   
415  407  Spain, Malta, under U.S./UK pressure, refuse t...   
416  408  Review: In Radioheadâ€™s â€˜A Moon Shaped Pool...   

                                           author  \
0                                   Darrell Lucus   
1                                 Daniel J. Flynn   
2                              Consortiumnews.com   
3                              

#Stemming:
#Stemming is the process of reducing a word to its Root word
#example: actor, actress, acting --> act



In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0      darrel lucu hous dem aid even see comey letter...
1      daniel j flynn flynn hillari clinton big woman...
2                 consortiumnew com truth might get fire
3      jessica purkiss civilian kill singl us airstri...
4      howard portnoy iranian woman jail fiction unpu...
                             ...                        
412    adam goldman eric lichtblau matt apuzzo comey ...
413             trump clinton choic impact global market
414            chines j stealth fighter debut sky zhuhai
415    j e dyer spain malta u uk pressur refus allow ...
416    jon parel review radiohead moon shape pool pat...
Name: content, Length: 417, dtype: object


In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire'
 'jessica purkiss civilian kill singl us airstrik identifi'
 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri'
 'daniel nussbaum jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight'
 'alissa j rubin beno hamon win french socialist parti presidenti nomin new york time'
 'excerpt draft script donald trump q ampa black church pastor new york time'
 'megan twohey scott shane back channel plan ukrain russia courtesi trump associ new york time'
 'aaron klein obama organ action partner soro link indivis disrupt trump agenda'
 'chri tomlinson bbc comedi sketch real housew isi caus outrag'
 'amando flavio russian research discov secret nazi militari base

In [ ]:
print(Y)

['1' '0' '1' '1' '1' '0' '1' '0' '0' '0' '0' '0' '1' '1' '' '' '' '' '' ''
 '' '' '0' '0' '1' '1' '0' '1' '0' '0' '1' '0' '1' '0' '1' '0' ''
 ' Karl Eikenberry. He' '0' '1' '0' '0' '0' '0' '1' '1' '0' '0' '0' '1'
 '0' '1' '0' '0' '1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0'
 '0' '1' '0' '1' '0' '0' '1' '1' '0' '1' '1' '1' '0' '0' '0' '0' '1' '1'
 '0' '0' '0' '0' '1' '1' '0' '0' '1' '0' '1' '0' '0' '1' '0' '1' '0' '0'
 '1' '0' '0' '0' '1' '0' '0' '1' '0' '1' '0' '1' '0' '0' '0' '0' '1' '0'
 '1' '0' '0' '0' '0' '0' '1' '1' '1' '0' '1' '1' '1' '0' '0' '0' '0' '1'
 '0' '0' '1' '1' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '0' '1' '0'
 '1' '1' '0' '0' '0' '0' '1' '0' '0' '1' '0' '1' '1' '1' '1' '0' '0' '0'
 '0' '1' '1' '0' '1' '1' '0' '1' '0' '1' '1' '1' '0' '0' '0' '0' '1' '0'
 '1' '0' '0' '1' '1' ''
 ' Brookover and Barry Bennett  â€”   all alumni of Dr. Ben Carsonâ€™s recently shuttered campaign  â€”   did not seem to have the appetite for the jugular that Cruzâ€™s team did.

In [ ]:
Y.shape

(417,)

# converting the textual data to numerical data

In [ ]:

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 2056)	0.29057543050239726
  (0, 1740)	0.305372841349992
  (0, 1190)	0.305372841349992
  (0, 1143)	0.29057543050239726
  (0, 1026)	0.2697196512808869
  (0, 936)	0.23854203761026
  (0, 669)	0.2790976669467932
  (0, 516)	0.29057543050239726
  (0, 489)	0.305372841349992
  (0, 403)	0.26179065201304996
  (0, 328)	0.3262286205715024
  (0, 46)	0.29057543050239726
  (1, 2183)	0.28951063143648337
  (1, 916)	0.2177760429546519
  (1, 755)	0.7003320187148809
  (1, 486)	0.29957676945855255
  (1, 383)	0.2288552535155262
  (1, 295)	0.35016600935744047
  (1, 253)	0.17244896427075856
  (1, 208)	0.28951063143648337
  (2, 2047)	0.42811749598404014
  (2, 1267)	0.44991916885144745
  (2, 809)	0.3666620318267885
  (2, 740)	0.38570762234810324
  (2, 426)	0.44991916885144745
  :	:
  (414, 356)	0.37407688904560576
  (415, 2064)	0.2742526237219079
  (415, 1852)	0.30790302872553155
  (415, 1693)	0.2545683986607086
  (415, 1616)	0.30790302872553155
  (415, 1614)	0.30790302872553155
  (415, 1533)	0.27425262372

#Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=3)

#Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

#Evaluation



In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)

#Accuracy Score for training dataset

In [ ]:
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.972972972972973


# Testing the dataset on test data

In [ ]:

X_test_prediction = model.predict(X_test)

# Accuracy score on the test data

In [ ]:
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.8095238095238095


#Making a Predictive System



In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['1']
The news is Fake


In [ ]:
print(Y_test[3])

1
